In [1]:
%pip install -U onnxruntime onnx gdown requests torch numpy torchvision


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import time

In [ ]:
import requests
import torch
import torch.nn as nn
import onnxruntime as ort
import numpy as np
import json
import io
import sys
import base64
from torch.utils.data import Dataset, DataLoader
from typing import Tuple
import pickle
import os
from torchvision import transforms

cwd = os.getcwd()
print('cwd: ', cwd)

class TaskDataset(Dataset):
    def __init__(self, imgs, transform=None):
        self.imgs = imgs
        self.transform = transform

    def __getitem__(self, index) -> Tuple[torch.Tensor]:
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        return img

    def __len__(self):
        return len(self.imgs)



In [ ]:
downloaded = False

In [ ]:
import gdown

ModelStealingPub_id = '1nGzKrLOrb2w5E2ZZPySRQz0WxN77DTmL'

if not downloaded:
    gdown.download(f'https://drive.google.com/uc?id={ModelStealingPub_id}', 'ModelStealingPub.pt', quiet=False)

In [ ]:
downloaded=True

In [ ]:
# Load the dataset
# dataset : TaskDataset = torch.load("/content/drive/MyDrive/ModelStealingPub.pt")
dataset : TaskDataset = torch.load("ModelStealingPub.pt")

In [ ]:
# REQUESTING NEW API
TOKEN = "76282151"  # Change this to your actual token

In [ ]:

response = requests.get("http://34.71.138.79:9090/stealing_launch", headers={"token": TOKEN})
answer = response.json()

print(answer)  # {"seed": "SEED", "port": PORT}
if 'detail' in answer:
    sys.exit(1)

# save the values
SEED = str(answer['seed'])
PORT = str(answer['port'])
print(SEED)
print(PORT)

In [ ]:
SEED = 4707839
PORT = 9002

In [ ]:
from copy import deepcopy


images = deepcopy(dataset.imgs)

In [ ]:
# Initialize a list to hold the subarrays
images1K = []

# Divide the array into 13 subarrays each containing 1000 elements
for i in range(0, len(images), 1000):
    images1K.append(images[i:i + 1000])

# Verify the result
for idx, image1K in enumerate(images1K):
    print(f"Subarray {idx + 1}: Length = {len(image1K)}")

In [ ]:
def model_stealing(images, port):
    endpoint = "/query"
    url = f"http://34.71.138.79:{port}" + endpoint
    image_data = []
    for img in images:
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format='PNG')
        img_byte_arr.seek(0)
        img_base64 = base64.b64encode(img_byte_arr.getvalue()).decode('utf-8')
        image_data.append(img_base64)

    payload = json.dumps(image_data)
    response = requests.get(url, files={"file": payload}, headers={"token": TOKEN})
    if response.status_code == 200:
        representation = response.json()["representations"]
        return representation
    else:
        raise Exception(
            f"Model stealing failed. Code: {response.status_code}, content: {response.json()}"
        )


In [ ]:
# Query the API
outputs = []
for batch in images1K:
    batch_outputs = model_stealing(batch, port=PORT)
    print("Batch done")
    outputs.extend(batch_outputs)
    print(len(outputs))
    time.sleep(65)

In [ ]:
# Save the output in a file
with open('out.pickle', 'wb') as handle:
    pickle.dump(outputs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# Restore the output from the file
with open('out.pickle', 'rb') as handle:
    outputs = pickle.load(handle)

print(len(outputs))

In [ ]:
# Custom function to convert grayscale to RGB
def convert_to_rgb(image):
    if image.mode == 'L':
        # Convert grayscale to RGB
        image = image.convert('RGB')
    return image

# Custom collate function to handle PIL images
def collate_tensor(batch):
    batch = [transforms.Resize((32, 32))(img) for img in batch]
    batch = [convert_to_rgb(img) for img in batch]
    batch = [transforms.ToTensor()(img) for img in batch]
    return torch.stack(batch)

Preparation of the Image Dataset for the Stolen Model

In [ ]:
from torch.utils.data import Dataset, DataLoader

# Custom dataset class
class ImageDataset(Dataset):
    def __init__(self, images, transform=None):
        self.images = images
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        if self.transform:
            image = self.transform(image)
        return image

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.Grayscale(),
    transforms.ToTensor()
])

# Create a dataset instance
stolen_dataset = ImageDataset(images=images, transform=transform,)

# Create a DataLoader
stolen_dataloader = DataLoader(dataset, batch_size=32, shuffle=False,collate_fn=collate_tensor)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

A CNN with 3 convolutional layers to replicate the model performance. MSE loss is chosen since it can produce the L2 differences

In [ ]:
class StolenModel(nn.Module):
    def __init__(self):
        super(StolenModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.fc1 = nn.Linear(256 * 4 * 4, 1024)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        return x

model = StolenModel().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

outputs_tensor = torch.tensor(outputs, dtype=torch.float32)


epochs = 10  
for epoch in range(epochs):
    total_loss = 0.0  
    for i, imgs in enumerate(stolen_dataloader):
        imgs = imgs.to(device)
        optimizer.zero_grad()
        outputs_pred = model(imgs)
        
        start_idx = i * imgs.size(0)
        end_idx = start_idx + imgs.size(0)
        outputs_subset = outputs_tensor[start_idx:end_idx].to(device)
        loss = criterion(outputs_pred, outputs_subset)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()  

    # Calculate average loss for the epoch
    avg_loss = total_loss / len(stolen_dataloader)
    print(f"Epoch [{epoch + 1}/{epochs}], Avg Loss: {avg_loss}")



In [ ]:
# Save the model as ONNX
path = 'stolen_model.onnx'
dummy_input = torch.randn(1, 3, 32, 32, device=device)
torch.onnx.export(model, dummy_input, path, export_params=True, input_names=["x"])

# Validate the model
with open(path, "rb") as f:
    model_data = f.read()
    try:
        stolen_model = ort.InferenceSession(model_data)
    except Exception as e:
        raise Exception(f"Invalid model, {e=}")
    try:
        out = stolen_model.run(None, {"x": np.random.randn(1, 3, 32, 32).astype(np.float32)})[0][0]
    except Exception as e:
        raise Exception(f"Some issue with the input, {e=}")
    assert out.shape == (1024,), "Invalid output shape"



In [ ]:
import requests

path = 'stolen_model.onnx'
# Send the model to the server
response = requests.post(f"http://34.71.138.79:9090/stealing", files={"file": open(path, "rb")}, headers={"token": TOKEN, "seed": str(SEED)})
print(response.json())

CODE ENDS HERE!!!
BELOW CELLS SHOWCASE THE WORKS WHICH DIDN'T GIVE GOOD RESULTS

Tried to prepare augmented dataset according to "Stolen Encoder" but it didn't give good results

In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image


class AugmentedImageDataset(Dataset):
    def __init__(self, images):
        self.images = images
        self.to_tensor = transforms.ToTensor()
        self.horizontal_flip = transforms.RandomHorizontalFlip(p=1.0)
        self.converter = convert_to_rgb,
        self.color = transforms.ColorJitter(hue=0.5, contrast=0.5, saturation = 0.5, brightness = 0.5 )

    def __len__(self):
        return len(self.images) * 3 

    def __getitem__(self, idx):
        original_idx = idx // 3
        aug_type = idx % 3
        image = self.images[original_idx]

        if aug_type == 0:
            # Original image
            img = image
        elif aug_type == 1:
            # Horizontal flip
            img = self.horizontal_flip(image)
        else:
            # Grayscale
            img = self.color(image)
        
        img = self.converter(img)
        img = self.to_tensor(img)
        
        return img

# Create the dataset
aug_dataset = AugmentedImageDataset(images)

# Create a DataLoader
aug_data_loader = DataLoader(aug_dataset, batch_size=32, shuffle=False)

Tried to prepare a Augmented Pair Dataset along with SimCLR but that didn't work either

In [ ]:
import torch.nn.functional as F
from torchvision.models import resnet34


# Custom dataset for augmented pairs
class AugmentedPairsDataset(Dataset):
    def __init__(self, images, transform=None):
        self.images = images
        self.transform = transform
        self.augmentations = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
        ])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        if self.transform:
            image1 = self.transform(image)
            image2 = self.transform(image)
            image2 = self.augmentations(image2)
        return image1, image2

# SimCLR model with ResNet34 encoder and projection head
class SimCLR(nn.Module):
    def __init__(self, encoder, proj_dim=1024):
        super(SimCLR, self).__init__()
        self.encoder = encoder
        self.encoder.fc = nn.Identity()
        self.projector = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, proj_dim)
        )

    def forward(self, x1, x2):
        z1 = self.projector(self.encoder(x1))
        z2 = self.projector(self.encoder(x2))
        return z1, z2

# Contrastive loss function
def contrastive_loss(z1, z2, temperature=0.5):
    z1 = F.normalize(z1, dim=1)
    z2 = F.normalize(z2, dim=1)

    similarity_matrix = torch.matmul(z1, z2.T) / temperature

    batch_size = z1.size(0)
    mask = torch.eye(batch_size, device=z1.device).bool()

    positives = similarity_matrix[mask].view(batch_size, -1)

    negatives = similarity_matrix[~mask].view(batch_size, -1)

    labels = torch.zeros(batch_size, device=z1.device, dtype=torch.long)
    logits = torch.cat([positives, negatives], dim=1)
    loss = F.cross_entropy(logits, labels)

    return loss

# Define transforms
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

aug_pairs_dataset = AugmentedPairsDataset(images, transform=transform)
aug_pairs_data_loader = DataLoader(aug_pairs_dataset, batch_size=32, shuffle=True)

aug_outputs = [torch.randn(128) for _ in range(len(images))] 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
aug_outputs_tensor = torch.stack(aug_outputs).to(device)

encoder = resnet34(pretrained=True)
model = SimCLR(encoder).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train(model, data_loader, aug_outputs_tensor, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0

        for batch_idx, (images1, images2) in enumerate(aug_pairs_data_loader):
            images1, images2 = images1.to(device), images2.to(device)
            optimizer.zero_grad()
            
            # Forward pass
            z1, z2 = model(images1, images2)

            start_idx = batch_idx * images1.size(0)
            end_idx = start_idx + images1.size(0)
            if end_idx > len(aug_outputs_tensor):
                end_idx = len(aug_outputs_tensor)
                start_idx = end_idx - images1.size(0)

            aug_outputs_subset = aug_outputs_tensor[start_idx:end_idx]

            aug_outputs_subset = aug_outputs_subset[:z1.size(0)]

            loss_contrastive = contrastive_loss(z1, z2)

            print(z1.size(),z2.size(),aug_outputs_subset)
            similarity_loss = F.mse_loss(z1, aug_outputs_subset) + F.mse_loss(z2, aug_outputs_subset)

            loss = loss_contrastive + similarity_loss
            print(loss)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            
            if batch_idx % 100 == 99:   
                print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{batch_idx + 1}/{len(data_loader)}], Loss: {running_loss / 100:.3f}')
                running_loss = 0.0

# Train the model
outputs_tensor = torch.tensor(outputs).to(device)
train(model, aug_pairs_data_loader, outputs_tensor, optimizer, num_epochs=10)

Tried MaskedAutoEncoder too with 3 convolutional layers. However, the results were terrible

In [ ]:
import torch.nn.functional as F

class MaskedAutoEncoder(nn.Module):
    def __init__(self):
        super(MaskedAutoEncoder, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.fc1 = nn.Linear(256 * 4 * 4, 1024)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()

        # Masking layer
        self.mask = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1024),
            nn.Sigmoid()  
        )

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        
        masked_output = self.mask(x)
        masked_output = masked_output * x  
        return masked_output


model = MaskedAutoEncoder().to(device)


def soft_nearest_neighbors_loss(outputs_pred, outputs_subset):

    outputs_pred_norm = F.normalize(outputs_pred, p=2, dim=1)
    outputs_subset_norm = F.normalize(outputs_subset, p=2, dim=1)

    cosine_similarity = torch.matmul(outputs_pred_norm, outputs_subset_norm.T)

    loss = 1 - cosine_similarity
    return loss.mean()

# Convert outputs to tensor
outputs_tensor = torch.tensor(outputs, dtype=torch.float32)


epochs = 25  
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(epochs):
    total_loss = 0.0  
    for i, imgs in enumerate(stolen_dataloader):
        imgs = imgs.to(device)
        optimizer.zero_grad()
        outputs_pred = model(imgs)
        start_idx = i * len(imgs)
        end_idx = min(start_idx + len(imgs), len(outputs))
        outputs_subset = outputs_tensor[start_idx:end_idx].to(device)
        loss = soft_nearest_neighbors_loss(outputs_pred, outputs_subset)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() 

    avg_loss = total_loss / len(stolen_dataloader)
    print(f"Epoch [{epoch + 1}/{epochs}], Avg Loss: {avg_loss}")
